# Trial for tweeting and Apis

In [516]:
import ccxt
import json
from datetime import datetime
import time
import tweepy

In [2]:
f = open('../keys.json','r')
api_keys = json.loads(f.read())
f.close()

In [3]:
pair = 'DOGE/BTC'

## Twitter Trial

In [ ]:
# Read keys
f = open('../keys.json','r')
api_keys = json.loads(f.read())
f.close()
twitter_keys = {'consumer_key':api_keys['twitter_keys']['consumer_key'],'consumer_secret':api_keys['twitter_keys']['consumer_secret'],'access_token_key':api_keys['twitter_keys']['access_token_key'],'access_token_secret': api_keys['twitter_keys']['access_token_secret']}

In [ ]:
# Use twitter API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
api = tweepy.API(auth)

In [ ]:
user = ['ArbitrageDaddy', 1351770767130673152]

In [ ]:
tweets = api.user_timeline(user_id=user[1], count=1,include_rts = True, exclude_replies = True, tweet_mode = 'extended', wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
for tweet in tweets:
    print (tweet.full_text)


## Kraken

In [ ]:
kraken_keys = {'api_key':api_keys['krakenapi_keys']['api_key'],'secret_key':api_keys['krakenapi_keys']['secret_key']}
k = ccxt.kraken({'apiKey':kraken_keys['api_key'], 'secret':kraken_keys['secret_key']})


In [ ]:
funcs =  k.has

In [ ]:
trades = k.fetchMyTrades(pair)

In [ ]:
sell = trades[-1]
buy = trades[-2]

In [ ]:
usdpair = k.fetchTicker('BTC/USD')
(usdpair['bid']+usdpair['ask'])/2

In [ ]:
btc_price = (usdpair['bid'] + usdpair['ask']) / 2

In [ ]:
print((sell['cost'] - buy['cost'] - buy['fee']['cost'] - sell['fee']['cost']) * (usdpair['bid']+usdpair['ask'])/2)


## Binance

In [4]:
bin_keys = {'api_key':api_keys['binance_keys']['api_key'],'secret_key':api_keys['binance_keys']['secret_key']}
b = ccxt.binance({'apiKey':bin_keys['api_key'], 'secret':bin_keys['secret_key']})

In [222]:
# tousd = 'DOGE/USDT'
pair = 'DOGE/GBP'
amount = 200

In [223]:
# buy  = b.create_order(pair,'market','buy',amount)
# sell = b.create_order(pair,'market','sell',amount)

In [91]:
tickers = b.fetchTickers()

gbp_tickers = sorted(filter(lambda x : x.endswith('/GBP'), list(tickers.keys())))
btc_tickers = sorted(filter(lambda x : x.endswith('/BTC') or x.startswith('BTC/'), list(tickers.keys())))
usd_tickers = sorted(filter(lambda x : x.endswith('/USD'), list(tickers.keys())))
usdt_tickers = sorted(filter(lambda x : x.endswith('/USDT'), list(tickers.keys())))
bnb_tickers = sorted(filter(lambda x : x.endswith('/BNB') or x.startswith('BNB/'), list(tickers.keys())))
print(len(gbp_tickers))
print(len(usdt_tickers))
print(len(btc_tickers))
print(len(bnb_tickers))

7
258
310
216


In [59]:
tickers = set(tickers.keys())

In [139]:
selling = 0.05375*5332.6+0.05376*243.6+0.05385*5912.5+0.05391*226.7+0.05392*1000.0+0.05395*4284.5
avg_sell = (0.05375*5332.6+0.05376*243.6+0.05385*5912.5+0.05391*226.7+0.05392*1000.0+0.05395*4284.5) \
            / (5332.6+243.6+5912.5+226.7+1000.0+4284.5)
print(avg_sell)

buying = 0.05338*16006.8 + 0.05338*993.2
avg_buy = buying / (16006.8+993.2)
print(avg_buy)

selling - buying

0.05384746280860475
0.05338


7.941483000000062

In [ ]:
# Convert the fee currency to the Buying currency
# e.g.
''' 
1.
Buy  DOGE/BTC -> fee in DOGE: nothing
Sell DOGE/BTC -> fee in BTC: convert DOGE/BTC

2.
Buy  DOGE/BTC -> fee in BNB: convert DOGE/BNB
Sell DOGE/BTC -> fee in BNB: convert DOGE/BNB

3.
Buy  DOGE/GBP -> fee in DOGE: 
Sell DOGE/GBP -> fee in GBP: convert DOGE/GBP

4.
Buy  DOGE/GBP -> fee in BNB: convert DOGE/BNB
Sell DOGE/GBP -> fee in BNB: convert DOGE/BNB

5.
Buy  BTC/GBP -> fee in BNB: convert DOGE/BNB
Sell BTC/GBP -> fee in BNB: convert DOGE/BNB
'''

In [111]:
ticker = 'BTC/GBP'
trades = b.fetchMyTrades(ticker,limit=6)
for i in range(len(trades)-1):
    sell_trade = trades[-1-i]
    buy_trade = trades[-2-i]
    assert sell_trade['side'] == 'sell' and buy_trade['side'] == 'buy'

    buy_volume = buy_trade['amount']

    fee_pair = [ticker.split('/')[0],buy_trade['fee']['currency']]
    fee_ticker = '/'.join(fee_pair)
    fee_ticker_1 = '/'.join(fee_pair[::-1])

    if fee_ticker in tickers:
        fee = b.fetchTicker(fee_ticker)
    elif fee_ticker_1 in tickers:
        fee = b.fetchTicker(fee_ticker_1)

    fee_price = fee['last']

    sell_volume = buy_volume - fee_price*buy_trade['fee']['cost']
    print(buy_volume , sell_volume)


0.0004 0.000399709149818


In [123]:
ticker = 'DOGE/BTC'
trades = b.fetchMyTrades(ticker)
print(trades)
for i in range(len(trades)-1):
    sell_trade = trades[-1-i]
    buy_trade = trades[-2-i]
    assert sell_trade['side'] == 'sell' and buy_trade['side'] == 'buy'

    buy_volume = buy_trade['amount']

    fee_pair = [ticker.split('/')[0],buy_trade['fee']['currency']]
    fee_ticker = '/'.join(fee_pair)
    fee_ticker_1 = '/'.join(fee_pair[::-1])

    if fee_ticker in tickers:
        fee = b.fetchTicker(fee_ticker)
    elif fee_ticker_1 in tickers:
        fee = b.fetchTicker(fee_ticker_1)

    fee_price = fee['last']

    sell_volume = buy_volume - fee_price*buy_trade['fee']['cost']
    print(buy_volume , sell_volume)

[{'info': {'symbol': 'DOGEBTC', 'id': 8165194, 'orderId': 48256635, 'orderListId': -1, 'price': '0.00000150', 'qty': '100.00000000', 'quoteQty': '0.00015000', 'commission': '0.10000000', 'commissionAsset': 'DOGE', 'time': 1612894510373, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, 'timestamp': 1612894510373, 'datetime': '2021-02-09T18:15:10.373Z', 'symbol': 'DOGE/BTC', 'id': '8165194', 'order': '48256635', 'type': None, 'side': 'buy', 'takerOrMaker': 'taker', 'price': 1.5e-06, 'amount': 100.0, 'cost': 0.00015000000000000001, 'fee': {'cost': 0.1, 'currency': 'DOGE'}}, {'info': {'symbol': 'DOGEBTC', 'id': 8165237, 'orderId': 48256772, 'orderListId': -1, 'price': '0.00000149', 'qty': '99.00000000', 'quoteQty': '0.00014751', 'commission': '0.00000015', 'commissionAsset': 'BTC', 'time': 1612894526623, 'isBuyer': False, 'isMaker': False, 'isBestMatch': True}, 'timestamp': 1612894526623, 'datetime': '2021-02-09T18:15:26.623Z', 'symbol': 'DOGE/BTC', 'id': '8165237', 'order': '48256

AssertionError: 

In [382]:
buys =  {'info': {'symbol': 'DOGEGBP', 'orderId': 201464, 'orderListId': -1, 'clientOrderId': 'x-R4BD3S82161c4186eb0818448f3800', 'transactTime': 1612986441371, 'price': '0.00000000', 'origQty': '17000.00000000', 'executedQty': '17000.00000000', 'cummulativeQuoteQty': '907.46000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '0.05338000', 'qty': '993.20000000', 'commission': '0.00042856', 'commissionAsset': 'BNB', 'tradeId': 34080}, {'price': '0.05338000', 'qty': '16006.80000000', 'commission': '0.00690701', 'commissionAsset': 'BNB', 'tradeId': 34081}]}, 'id': '201464', 'clientOrderId': 'x-R4BD3S82161c4186eb0818448f3800', 'timestamp': 1612986441371, 'datetime': '2021-02-10T19:47:21.371Z', 'lastTradeTimestamp': None, 'symbol': 'DOGE/GBP', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'buy', 'price': 0.053380000000000004, 'stopPrice': None, 'amount': 17000.0, 'cost': 907.4599999999999, 'average': 0.05338, 'filled': 17000.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 0.00733557, 'currency': 'BNB'}, 'trades': [{'info': {'price': '0.05338000', 'qty': '993.20000000', 'commission': '0.00042856', 'commissionAsset': 'BNB', 'tradeId': 34080}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05338, 'amount': 993.2, 'cost': 53.017016, 'fee': {'cost': 0.00042856, 'currency': 'BNB'}}, {'info': {'price': '0.05338000', 'qty': '16006.80000000', 'commission': '0.00690701', 'commissionAsset': 'BNB', 'tradeId': 34081}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05338, 'amount': 16006.8, 'cost': 854.4429839999999, 'fee': {'cost': 0.00690701, 'currency': 'BNB'}}]}
sells = {'info': {'symbol': 'DOGEGBP', 'orderId': 202131, 'orderListId': -1, 'clientOrderId': 'x-R4BD3S827fca1b949f88d053b1631e', 'transactTime': 1612986743065, 'price': '0.00000000', 'origQty': '16999.90000000', 'executedQty': '16999.90000000', 'cummulativeQuoteQty': '915.40148300', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '0.05395000', 'qty': '4284.50000000', 'commission': '0.00187074', 'commissionAsset': 'BNB', 'tradeId': 34168}, {'price': '0.05392000', 'qty': '1000.00000000', 'commission': '0.00043638', 'commissionAsset': 'BNB', 'tradeId': 34169}, {'price': '0.05391000', 'qty': '226.70000000', 'commission': '0.00009891', 'commissionAsset': 'BNB', 'tradeId': 34170}, {'price': '0.05385000', 'qty': '5912.50000000', 'commission': '0.00257680', 'commissionAsset': 'BNB', 'tradeId': 34171}, {'price': '0.05376000', 'qty': '243.60000000', 'commission': '0.00010598', 'commissionAsset': 'BNB', 'tradeId': 34172}, {'price': '0.05375000', 'qty': '5332.60000000', 'commission': '0.00231975', 'commissionAsset': 'BNB', 'tradeId': 34173}]}, 'id': '202131', 'clientOrderId': 'x-R4BD3S827fca1b949f88d053b1631e', 'timestamp': 1612986743065, 'datetime': '2021-02-10T19:52:23.065Z', 'lastTradeTimestamp': None, 'symbol': 'DOGE/GBP', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'sell', 'price': 0.05384746280860475, 'stopPrice': None, 'amount': 16999.9, 'cost': 915.4014830000001, 'average': 0.05384746280860476, 'filled': 16999.9, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 0.0074085600000000015, 'currency': 'BNB'}, 'trades': [{'info': {'price': '0.05395000', 'qty': '4284.50000000', 'commission': '0.00187074', 'commissionAsset': 'BNB', 'tradeId': 34168}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05395, 'amount': 4284.5, 'cost': 231.148775, 'fee': {'cost': 0.00187074, 'currency': 'BNB'}}, {'info': {'price': '0.05392000', 'qty': '1000.00000000', 'commission': '0.00043638', 'commissionAsset': 'BNB', 'tradeId': 34169}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05392, 'amount': 1000.0, 'cost': 53.92, 'fee': {'cost': 0.00043638, 'currency': 'BNB'}}, {'info': {'price': '0.05391000', 'qty': '226.70000000', 'commission': '0.00009891', 'commissionAsset': 'BNB', 'tradeId': 34170}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05391, 'amount': 226.7, 'cost': 12.221397, 'fee': {'cost': 9.891e-05, 'currency': 'BNB'}}, {'info': {'price': '0.05385000', 'qty': '5912.50000000', 'commission': '0.00257680', 'commissionAsset': 'BNB', 'tradeId': 34171}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05385, 'amount': 5912.5, 'cost': 318.388125, 'fee': {'cost': 0.0025768, 'currency': 'BNB'}}, {'info': {'price': '0.05376000', 'qty': '243.60000000', 'commission': '0.00010598', 'commissionAsset': 'BNB', 'tradeId': 34172}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05376, 'amount': 243.6, 'cost': 13.095936, 'fee': {'cost': 0.00010598, 'currency': 'BNB'}}, {'info': {'price': '0.05375000', 'qty': '5332.60000000', 'commission': '0.00231975', 'commissionAsset': 'BNB', 'tradeId': 34173}, 'timestamp': None, 'datetime': None, 'symbol': 'DOGE/GBP', 'id': None, 'order': None, 'type': None, 'side': None, 'takerOrMaker': None, 'price': 0.05375, 'amount': 5332.6, 'cost': 286.62725, 'fee': {'cost': 0.00231975, 'currency': 'BNB'}}]}

In [497]:
# ticker = 'DOGE/GBP'
# ticker = 'DOGE/BTC'
# amount = 500
ticker, amount = 'BTC/GBP', 0.00033

In [498]:
# buy  = b.create_order(ticker,'market','buy',amount)

In [499]:
buy_trade = buy
buy_volume = buy_trade['amount']

In [500]:
# To calculate the correct volume to sell
# buy_volume = 200

trades = b.fetchMyTrades(ticker,limit = 20)
buy_id = buy_trade['info']['orderId']
# buy_id = 48605894

if buy_trade['fee']['currency'] == 'BNB':
    sell_volume = buy_volume
else:
    
    # Converting fee currency to buy currency
    ticker_pair = ticker.split('/')
    if ticker_pair[0] != buy_trade['fee']['currency']:
        fee_pair = [ticker_pair[0], buy_trade['fee']['currency']]
        fee_ticker = '/'.join(fee_pair)
        if fee_ticker not in tickers:
            fee_ticker = '/'.join(fee_pair[::-1])
        fee = b.fetchTicker(fee_ticker)
        fee_price = (fee['bid'] + fee['ask']) /2
        
        sell_volume = buy_volume - fee_price * buy_trade['fee']['cost']
    
    # When fee currency is the same as the buy currency
    else:
        sell_volume = buy_volume - buy_trade['fee']['cost']
        
print(sell_volume)

0.00033


In [501]:
# sell = b.create_order(ticker,'market','sell',0.000000001)

sell_trade = sell
sell_id = sell_trade['info']['orderId']

In [502]:
# To print and calculate profits

buy_prices, sell_prices = [], []
trades = b.fetchMyTrades(ticker,limit = 20)
for trade in trades[::-1]:
    if buy_id == trade['info']['orderId']:
        buy_prices.append({'amount':trade['amount'],'cost':trade['cost'],'fee':trade['fee']})
    elif sell_id == trade['info']['orderId']:
        sell_prices.append({'amount':trade['amount'],'cost':trade['cost'],'fee':trade['fee']}) # Actual return uses fills

buy_fee = sum([x['fee']['cost'] for x in buy_prices])
sell_fee = sum([x['fee']['cost'] for x in sell_prices])        

# Put in some error handling
if buy_trade['fee']['currency'] == 'BNB':
    bnb_dollar = b.fetch_ticker('BNB/USDT')
    bnb_price = (bnb_dollar['bid'] + bnb_dollar['ask']) / 2
    buy_fee_dollar = buy_fee * bnb_price
    if sell_trade['fee']['currency'] == 'BNB':
        sell_fee_dollar = sell_fee * bnb_price
else:
    buy_crypto_dollar = b.fetch_ticker(buy_trade['fee']['currency']+'/USDT')
    sell_crypto_dollar = b.fetch_ticker(sell_trade['fee']['currency']+'/USDT')
    buy_fee_price = (buy_crypto_dollar['bid']+buy_crypto_dollar['ask'])/2
    sell_fee_price = (sell_crypto_dollar['bid']+sell_crypto_dollar['ask'])/2
    
    buy_fee_dollar = buy_fee_price * buy_fee
    sell_fee_dollar = sell_fee_price * sell_fee

ticker_pair = ticker.split('/')
ticker_info = b.fetch_ticker(ticker_pair[1]+'/'+'USDT')

print(sell_trade['cost'], buy_trade['cost'])
print((sell_trade['cost'] - buy_trade['cost'])*(ticker_info['bid'] + ticker_info['ask'])\
      / 2 - sell_fee_dollar - buy_fee_dollar)

10.7217495 10.7272737
-0.029760595245999964


In [503]:
ticker_info

{'symbol': 'GBP/USDT',
 'timestamp': 1613020636056,
 'datetime': '2021-02-11T05:17:16.056Z',
 'high': 1.3813,
 'low': 1.3689,
 'bid': 1.3767,
 'bidVolume': 49951.36,
 'ask': 1.3768,
 'askVolume': 2382.26,
 'vwap': 1.37632234,
 'open': 1.3761,
 'close': 1.3767,
 'last': 1.3767,
 'previousClose': 1.3763,
 'change': 0.0006,
 'percentage': 0.044,
 'average': None,
 'baseVolume': 34519094.06,
 'quoteVolume': 47509400.1588,
 'info': {'symbol': 'GBPUSDT',
  'priceChange': '0.00060000',
  'priceChangePercent': '0.044',
  'weightedAvgPrice': '1.37632234',
  'prevClosePrice': '1.37630000',
  'lastPrice': '1.37670000',
  'lastQty': '152.74000000',
  'bidPrice': '1.37670000',
  'bidQty': '49951.36000000',
  'askPrice': '1.37680000',
  'askQty': '2382.26000000',
  'openPrice': '1.37610000',
  'highPrice': '1.38130000',
  'lowPrice': '1.36890000',
  'volume': '34519094.06000000',
  'quoteVolume': '47509400.15880000',
  'openTime': 1612934236056,
  'closeTime': 1613020636056,
  'firstId': 4300062,
  

In [504]:
sell_fee_dollar, buy_fee_dollar

(0.011077576447999999, 0.011077576447999999)

In [508]:
buy_trade['symbol']

'BTC/GBP'

In [513]:
buy_trade['info']['fills']

[{'price': '32506.89000000',
  'qty': '0.00033000',
  'commission': '0.00008704',
  'commissionAsset': 'BNB',
  'tradeId': 5257017}]

In [526]:
avg_price = sum([float(x['price']) * float(x['qty']) for x in buy_trade['info']['fills']])/sum([float(x['qty']) for x in buy_trade['info']['fills']])
to_print = '\nBought %s of %s at %s with %s %s of fees on %s' % (buy_trade['amount'], buy_trade['symbol'], avg_price, buy_trade['fee']['cost'], buy_trade['fee']['currency'], datetime.now().strftime('%b %d - %H:%M:%S'))

print(to_print)

avg_price = sum([float(x['price']) * float(x['qty']) for x in sell_trade['info']['fills']])/sum([float(x['qty']) for x in sell_trade['info']['fills']])
to_print = '\nSold %s of %s at %s with %s %s of fees on %s' % (sell_trade['amount'], sell_trade['symbol'], avg_price, sell_trade['fee']['cost'], sell_trade['fee']['currency'], datetime.now().strftime('%b %d - %H:%M:%S'))
print(to_print)
                                  


Bought 0.00033 of BTC/GBP at 32506.89 with 8.704e-05 BNB of fees on Feb 10 - 21:37:59

Sold 0.00033 of BTC/GBP at 32490.149999999998 with 8.704e-05 BNB of fees on Feb 10 - 21:37:59


In [545]:
ticker = 'DOGE/GBP'

In [548]:
try:
    b.create_order(ticker,'market','sell',300)
except Exception as e:
    error = e
    print(e)

binance Account has insufficient balance for requested action.


In [544]:
if 'MIN_NOTIONAL' in str(error):
    print('hi')

hi


In [505]:
buy_trade

{'info': {'symbol': 'BTCGBP',
  'orderId': 91381602,
  'orderListId': -1,
  'clientOrderId': 'x-R4BD3S82dcfce58c682198b06e2bac',
  'transactTime': 1613020635565,
  'price': '0.00000000',
  'origQty': '0.00033000',
  'executedQty': '0.00033000',
  'cummulativeQuoteQty': '10.72727370',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'fills': [{'price': '32506.89000000',
    'qty': '0.00033000',
    'commission': '0.00008704',
    'commissionAsset': 'BNB',
    'tradeId': 5257017}]},
 'id': '91381602',
 'clientOrderId': 'x-R4BD3S82dcfce58c682198b06e2bac',
 'timestamp': 1613020635565,
 'datetime': '2021-02-11T05:17:15.565Z',
 'lastTradeTimestamp': None,
 'symbol': 'BTC/GBP',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 32506.89,
 'stopPrice': None,
 'amount': 0.00033,
 'cost': 10.7272737,
 'average': 32506.89,
 'filled': 0.00033,
 'remaining': 0.0,
 'status': 'closed',
 'fee': {'cost': 8.704e-05, 'currency': 'BN

In [494]:
sell_trade

{'info': {'symbol': 'BTCGBP',
  'orderId': 91379781,
  'orderListId': -1,
  'clientOrderId': 'x-R4BD3S824a9ef46133608750313593',
  'transactTime': 1613020488035,
  'price': '0.00000000',
  'origQty': '0.00033000',
  'executedQty': '0.00033000',
  'cummulativeQuoteQty': '10.70864095',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'fills': [{'price': '32454.14000000',
    'qty': '0.00000500',
    'commission': '0.00000132',
    'commissionAsset': 'BNB',
    'tradeId': 5256915},
   {'price': '32450.37000000',
    'qty': '0.00032500',
    'commission': '0.00008619',
    'commissionAsset': 'BNB',
    'tradeId': 5256916}]},
 'id': '91379781',
 'clientOrderId': 'x-R4BD3S824a9ef46133608750313593',
 'timestamp': 1613020488035,
 'datetime': '2021-02-11T05:14:48.035Z',
 'lastTradeTimestamp': None,
 'symbol': 'BTC/GBP',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'sell',
 'price': 32450.42712121212,
 'stopPrice': None,
 'amount': 0